In [337]:
import warnings
#warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import time

In [338]:
STAT_COLUMNS = [
    'st', 'carav', 'g', 
    'cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int',
    'rush_att', 'rush_yds', 'rush_tds',
    'rec', 'rec_yds', 'rec_tds', 'tkl',
    'def_int'
]

POSITION_DEPENDENT_FEATURES = [
    'cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int', 'rush_att', 'rush_yds', 'rush_tds', 'rec', 'rec_yds', 'rec_tds', 'tkl', 'def_int', 'sk', 
]


In [339]:
# import and preview dataset
data = pd.read_csv("nfl_draft.csv")
data.head()

,column_a,player_id,year,rnd,pick,tm,player,hof,pos,position_standard,...,rush_att,rush_yds,rush_tds,rec,rec_yds,rec_tds,tkl,def_int,sk,college_univ
0,2015Jameis Winston,WinsJa00,2015,1,1,TAM,Jameis Winston,No,QB,QB,...,86.0,311.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,Florida St.
1,2015Marcus Mariota,MariMa01,2015,1,2,TEN,Marcus Mariota,No,QB,QB,...,79.0,516.0,4.0,1.0,41.0,1.0,NaN,NaN,NaN,Oregon
2,2015Dante Fowler,FowlDa00,2015,1,3,JAX,Dante Fowler,No,OLB,LB,...,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,2.5,Florida
3,2015Amari Cooper,CoopAm00,2015,1,4,OAK,Amari Cooper,No,WR,WR,...,3.0,-3.0,0.0,134.0,1970.0,9.0,NaN,NaN,NaN,Alabama
4,2015Brandon Scherff,ScheBr00,2015,1,5,WAS,Brandon Scherff,No,T,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Iowa


In [340]:
# frequency of null entries in each feature
data.isnull().sum()

column_a                0
player_id            1219
year                    0
rnd                     0
pick                    0
tm                      0
player                  0
hof                     0
pos                     0
position_standard       0
first4av                0
age                  1245
to                   1382
ap1                     0
pb                      0
st                      0
carav                1382
drav                 2165
g                    1415
cmp                  7841
pass_att             7841
pass_yds             7841
pass_td              7841
pass_int             7841
rush_att             6789
rush_yds             6789
rush_tds             6789
rec                  6264
rec_yds              6264
rec_tds              6264
tkl                  4324
def_int              6819
sk                   4388
college_univ         3430
dtype: int64

In [341]:
#Many features have null values. We suspect that the reason is that some
#positions do not do things that other positions do. For example, a linebacker
#will likely never throw a football, so they will have null for cmp
#(completions)

#let's verify and check how many QB's have null completions vs total amount of
#null completions
null_cmps = data[data['cmp'].isnull()]
null_cmps


,column_a,player_id,year,rnd,pick,tm,player,hof,pos,position_standard,...,rush_att,rush_yds,rush_tds,rec,rec_yds,rec_tds,tkl,def_int,sk,college_univ
2,2015Dante Fowler,FowlDa00,2015,1,3,JAX,Dante Fowler,No,OLB,LB,...,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,2.5,Florida
3,2015Amari Cooper,CoopAm00,2015,1,4,OAK,Amari Cooper,No,WR,WR,...,3.0,-3.0,0.0,134.0,1970.0,9.0,NaN,NaN,NaN,Alabama
4,2015Brandon Scherff,ScheBr00,2015,1,5,WAS,Brandon Scherff,No,T,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Iowa
5,2015Leonard Williams,WillLe02,2015,1,6,NYJ,Leonard Williams,No,DE,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,56.0,NaN,9,USC
6,2015Kevin White,WhitKe00,2015,1,7,CHI,Kevin White,No,WR,WR,...,1.0,9.0,0.0,19.0,187.0,0.0,NaN,NaN,NaN,West Virginia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8430,1985Raymond Polk,NaN,1985,12,332,RAI,Raymond Polk,No,DB,DB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oklahoma St.,NaN
8431,1985Paul Woodside,NaN,1985,12,333,BUF,Paul Woodside,No,K,K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Virginia,NaN
8432,1985Dan Lynch,NaN,1985,12,334,DEN,Dan Lynch,No,G,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington St.,NaN
8433,1985Ray Noble,NaN,1985,12,335,MIA,Ray Noble,No,DB,DB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,California,NaN


In [342]:
null_cmps.loc[null_cmps['pos'] == 'QB']

# as we can see, very few QB positions have null values for cmp while many non
# QB positions do. Because of this, we think it is best to impute a value of 0 
# for the position-dependent features completions, rush attempts, touchdowns, 
# etc) 

,column_a,player_id,year,rnd,pick,tm,player,hof,pos,position_standard,...,rush_att,rush_yds,rush_tds,rec,rec_yds,rec_tds,tkl,def_int,sk,college_univ
74,2015Garrett Grayson,GrayGa00,2015,3,75,NOR,Garrett Grayson,No,QB,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Colorado St.
418,2014Aaron Murray,MurrAa00,2014,5,163,KAN,Aaron Murray,No,QB,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Georgia
438,2014David Fales,FaleDa00,2014,6,183,CHI,David Fales,No,QB,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Jose St.
449,2014Keith Wenning,WennKe00,2014,6,194,BAL,Keith Wenning,No,QB,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ball St.
468,2014Tajh Boyd,BoydTa00,2014,6,213,NYJ,Tajh Boyd,No,QB,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clemson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8330,1985Steve Calabria,NaN,1985,9,232,TAM,Steve Calabria,No,QB,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Colgate,NaN
8332,1985Paul Berner,NaN,1985,9,234,SDG,Paul Berner,No,QB,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pacific,NaN
8367,1985Buddy Funck,NaN,1985,10,269,DEN,Buddy Funck,No,QB,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New Mexico,NaN
8375,1985John Conner,NaN,1985,10,277,SEA,John Conner,No,QB,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona,NaN


In [343]:
data[POSITION_DEPENDENT_FEATURES] = data[POSITION_DEPENDENT_FEATURES].fillna(0)
data.isnull().sum()

column_a                0
player_id            1219
year                    0
rnd                     0
pick                    0
tm                      0
player                  0
hof                     0
pos                     0
position_standard       0
first4av                0
age                  1245
to                   1382
ap1                     0
pb                      0
st                      0
carav                1382
drav                 2165
g                    1415
cmp                     0
pass_att                0
pass_yds                0
pass_td                 0
pass_int                0
rush_att                0
rush_yds                0
rush_tds                0
rec                     0
rec_yds                 0
rec_tds                 0
tkl                     0
def_int                 0
sk                      0
college_univ         3430
dtype: int64

In [344]:
#unneeded things like name/player id, team, etc. Does not help with our prediction
data.drop(['column_a','player_id','tm', 'hof', 'college_univ'], axis=1, inplace=True)

In [345]:
data['position_standard'].value_counts()
#combine C and LS and we're good

DB    1569
LB    1124
WR    1049
RB     809
DE     731
DT     640
T      619
G      542
TE     496
QB     398
C      225
K       78
FB      77
P       77
LS       1
Name: position_standard, dtype: int64

In [346]:
data.columns

Index(['year', 'rnd', 'pick', 'player', 'pos', 'position_standard', 'first4av',
       'age', 'to', 'ap1', 'pb', 'st', 'carav', 'drav', 'g', 'cmp', 'pass_att',
       'pass_yds', 'pass_td', 'pass_int', 'rush_att', 'rush_yds', 'rush_tds',
       'rec', 'rec_yds', 'rec_tds', 'tkl', 'def_int', 'sk'],
      dtype='object')

In [347]:
# data.head()
# data[data['first4av'] > 1]['first4av'].value_counts()
# data[data['first4av'] == 0]['rnd'].value_counts()
data['']

#remove column_a, player_id, tm, player, hof, pos, 
#label is in rnd column
# output = data.corr()
# output[['drav', 'carav']]
#correlation between first4av and rnd is -0.48 AND it's not clear what first4av represents
# data.corr()

'''
Index(['year', 'rnd', 'pick', 'player', 'pos', 'position_standard', 'first4av',
       'age', 'to', 'ap1', 'pb', 'st', 'carav', 'drav', 'g', 'cmp', 'pass_att',
       'pass_yds', 'pass_td', 'pass_int', 'rush_att', 'rush_yds', 'rush_tds',
       'rec', 'rec_yds', 'rec_tds', 'tkl', 'def_int', 'sk'],
      dtype='object')
'''

KeyError: ''

In [ ]:
# print(data['rnd'].corr(data['drav']))
# print(data['rnd'].corr(data['carav']))
#drop drav b/c corr with rnd lower than carav and they are similar
data[['st', 'g', 'cmp', 'pass_att',
       'pass_yds', 'pass_td', 'pass_int', 'rush_att', 'rush_yds', 'rush_tds',
       'rec', 'rec_yds', 'rec_tds', 'tkl', 'def_int', 'sk']].head()
# data['rnd'].corr(data['to'])
#future data to remove: ap1 (Deals with all pro designation), pb (Deals with Pro Bowler designation), hof (hall of fame designation)

,st,g,cmp,pass_att,pass_yds,pass_td,pass_int,rush_att,rush_yds,rush_tds,rec,rec_yds,rec_tds,tkl,def_int,sk
0,2,26.0,540.0,913.0,6722.0,42.0,25.0,86.0,311.0,7.0,0.0,0.0,0.0,0.0,0.0,0
1,2,23.0,458.0,725.0,5590.0,42.0,18.0,79.0,516.0,4.0,1.0,41.0,1.0,0.0,0.0,0
2,0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,2.5
3,1,26.0,0.0,0.0,0.0,0.0,0.0,3.0,-3.0,0.0,134.0,1970.0,9.0,0.0,0.0,0
4,2,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
data.drop(['ap1', 'pb'], axis=1, inplace=True)

In [ ]:
data[['first4av', 'rnd']]

,first4av,rnd
0,13,1
1,9,1
2,0,1
3,9,1
4,7,1
...,...,...
8430,0,12
8431,0,12
8432,0,12
8433,0,12


In [ ]:
def transform(x):
  if(x['year'] <= 1993): #rounds before 1993 did not have 32 rounds per pick. We should standardize to today's standard
    x['rnd'] = 1 + int(x['pick'] / 32)
  return x

data = data.apply(func=transform, axis=1, result_type='broadcast')
labels = data['rnd']
#labels[labels > 3] = 4

KeyError: 'pick'

In [ ]:
# importing the module
import collections


# using Counter to find frequency of elements
frequency = collections.Counter(labels)

# printing the frequency
print(dict(frequency))


{1: 970, 2: 985, 3: 1041, 4: 1075, 5: 1063, 6: 1127, 7: 1284, 8: 257, 9: 256, 10: 256, 11: 121}


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

dtc = DecisionTreeClassifier(criterion="entropy")
dtc.fit(X_train, y_train)

from sklearn.naive_bayes import GaussianNB

accuracies = cross_val_score(GaussianNB(), X=features, y=labels.astype('int'), cv=10)

print('Accuracy of model:', accuracy_score(y_test, dtc.predict(X_test)))
print('Accuracy of model (gaussianNB):', sum(accuracies)/len(accuracies))


NameError: name 'accuracy_score' is not defined